In [24]:
## Packages
using LinearAlgebra
using Printf

include("BasicFunctions.jl")

## Translation to POV-Ray code
function translate2pov(x::Real)
    return @sprintf "%1.24f" x
end

function translate2pov(x::Array{T,1}) where T <: Real
    y=repr(translate2pov.(x))
    y=replace(y,"\""=>"")
    return "<"*y[2:end-1]*">"
end

## Types
abstract type JuliRay end
abstract type Object <: JuliRay end

include("PrimitiveObject.jl")
include("CSG.jl")
include("Transformation.jl")
include("Pigment.jl")
include("CompoundObject.jl")
include("Camera.jl")

# Rendering
function render(object::Object;name="new", index::Int=0, camera::Camera=LngLatCamera(), lights::Array{T,1}=Light[]) where T <: Light
    width=camera.width
    height=camera.height
    
    if(0<index<1000000)
        Index="_"*(@sprintf "%06d" index)
    elseif(index==0)
        Index=""
    else
        error("index must be non-negative and less than 1000000")
    end
    if(endswith(name,".pov"))
        Name=name[1:end-4]*Index*".pov"
    else
        Name=name*Index*".pov"
    end
    
    str="#version 3.7;\nglobal_settings{assumed_gamma 1.0}\n"
    str=str*translate2pov(camera)*"\n"
    str=str*(join(translate2pov.(lights)))
    str=str*"background{rgb<1,1,1>}"*"\n"
    str=str*translate2pov(object)*"\n"
    io=open(Name,"w")
    write(io,str)
    close(io)

    str="Width=$width\nHeight=$height\nAntialias=On\n"
    io=open("povray.ini","w")
    write(io,str)
    close(io)
    
    run(`povray $Name`)
end

render (generic function with 1 method)

In [25]:
a=rgbColor(Sphere([2,-1,-1]/3,0.6),RGB(1,0,0))
b=rgbColor(Sphere([-1,2,-1]/3,0.6),RGB(0.5,0,0.5))
c=rgbColor(Sphere([-1,-1,2]/3,0.6),RGB(0,1,0))
abc=csgMerge(a,b,c)
render(Transparent(abc,FT(0.1,0.2)),name="julia000",index=0,camera=LngLatCamera(lng=π/5,lat=π/5,pers=0.5,zoom=0.3))

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

Process(`povray julia000.pov`, ProcessExited(0))

In [26]:
ls=[PointLight([2,2,2],RGB(0,1,0))]
render(Transparent(abc,FT(0.1,0.2)),name="julia002",index=0,camera=LngLatCamera(lng=π/5,lat=π/5,pers=0.5,zoom=0.3),lights=ls)

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

Process(`povray julia002.pov`, ProcessExited(0))

In [27]:
vertices=[[i,j,k] for i ∈ [1,-1], j ∈ [1,-1], k ∈ [1,-1]]
edges=[[([[i,j,1],[i,j,2]],[[1,i,j],[2,i,j]],[[j,1,i],[j,2,i]])[k] for i ∈ 1:2, j ∈ 1:2, k ∈ 1:3]...]
faces=[([[1,1,i],[1,2,i],[2,2,i],[2,1,i]],[[i,1,1],[i,1,2],[i,2,2],[i,2,1]],[[1,i,1],[1,i,2],[2,i,2],[2,i,1]])[j] for i ∈ 1:2, j ∈ 1:3]
V=csgUnion((p->Sphere(p,0.05)).([vertices...]))
E=csgUnion([Cylinder(vertices[I...],vertices[J...],0.01) for (I,J) in [edges...]])
F=csgUnion([Polygon(vertices[I...],vertices[J...],vertices[K...],vertices[L...]) for (I,J,K,L) in [faces...]])
render(rgbftColor(csgUnion(V,E,F),RGB(1,0,0),FT(0.2,0.2)),camera=LngLatCamera(lng=π/5,lat=π/5,pers=0.5,zoom=0.2))

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

Process(`povray new.pov`, ProcessExited(0))

In [28]:
render(ParallelTranslation(rgbftColor(csgUnion(V,E,F),RGB(1,0,0),FT(0.2,0.2)),[1,0,0]),camera=LngLatCamera(lng=π/5,lat=π/5,pers=0.5,zoom=0.2),name="ho")

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

Process(`povray ho.pov`, ProcessExited(0))

In [29]:
render(Scaling(rgbftColor(csgUnion(V,E,F),RGB(1,0,0),FT(0.2,0.2)),0.5),camera=LngLatCamera(lng=π/5,lat=π/5,pers=0.5,zoom=0.2),name="hoi")

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

Process(`povray hoi.pov`, ProcessExited(0))

In [6]:
vertices=[[[i,j,k] for i ∈ [1,-1], j ∈ [1,-1], k ∈ [1,-1]]...]
edges=[[[1+i+j,2+i+j] for i ∈ [0,2], j ∈ [0,4]]...]
edges=vcat(edges,[[[1+i+j,3+i+j] for i ∈ [0,1], j ∈ [0,4]]...])
edges=vcat(edges,[[[1+i+j,5+i+j] for i ∈ [0,1], j ∈ [0,2]]...])
V=csgUnion((p->Sphere(p,0.05)).([vertices...]))
E=csgUnion([Cylinder(vertices[I[1]],vertices[I[2]],0.03) for I in edges])
# F=csgUnion([Polygon(vertices[I...],vertices[J...],vertices[K...],vertices[L...]) for (I,J,K,L) in [faces...]])
render(rgbftColor(csgUnion(V,E,F),RGB(1,0,1),FT(0.2,0.2)),camera=LngLatCamera(lng=π/5,lat=π/5,pers=0.5,zoom=0.25))

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

Process(`povray new.pov`, ProcessExited(0))

In [71]:
[faces...]

6-element Array{Array{Array{Int64,1},1},1}:
 [[1, 1, 1], [1, 2, 1], [2, 2, 1], [2, 1, 1]]
 [[1, 1, 2], [1, 2, 2], [2, 2, 2], [2, 1, 2]]
 [[1, 1, 1], [1, 1, 2], [1, 2, 2], [1, 2, 1]]
 [[2, 1, 1], [2, 1, 2], [2, 2, 2], [2, 2, 1]]
 [[1, 1, 1], [1, 1, 2], [2, 1, 2], [2, 1, 1]]
 [[1, 2, 1], [1, 2, 2], [2, 2, 2], [2, 2, 1]]

In [8]:
csgUnion

csgUnion

In [444]:
function randonS2()
    p=(2*rand(3)).-1
    while(norm(p)>1.0)
        p=(2*rand(3)).-1
    end
    return normalize(p)
end

m=100
n=100
for j in 0:n
    t=j/n
#     red=rgbColor(csgDifference(Sphere([0,0,0],1),AffineTransform(Box([-3,-3,-0.1],[3,3,0.1])),rotatematrix()),RGB(1,0,0))
    
    violet=[]
    for i in 1:m
        push!(violet,rgbColor(Polygon(randonS2(),randonS2(),randonS2()),RGB(0.5+rand()/8-1/16,rand()/8,0.5+rand()/8-1/16)))
    end
    violet=csgUnion(violet)
    red=csgUnion(violet)

    green=[]
    for i in 1:m
        push!(green,rgbColor(Polygon(randonS2(),randonS2(),randonS2()),RGB(rand()/8,1-rand()/8,rand()/8)))
    end
    green=csgUnion(green)
    
    red=AffineTransform(red,[0.6 0 0;0 0.6 0;0 0 0.6],[2,-1,-1]/3)
    violet=AffineTransform(violet,[0.6 0 0;0 0.6 0;0 0 0.6],[-1,2,-1]/3)
    green=AffineTransform(green,[0.6 0 0;0 0.6 0;0 0 0.6],[-1,-1,2]/3)

    render(csgUnion(red,violet,green),index=j+1,camera=LngLatCamera(lng=π/5+j/100,lat=π/6,pers=0.001,zoom=0.3))
end

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

File 'new_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000001.pov' line 6: Parse Warning: Polygon not closed. Closing it.

Rendered 250000 of 250000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 500 x 500
----------------------------------------------------------------------------
Pixels:           266000   Samples:          138744   Smpls/Pxl: 0.52
Rays:             404744   Saved:                 0   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Polygon                        7657509          647688      8.46
Bounding Box                  44813835        15964830     35.62


----------------------------------------------------------------------------
Shadow Ray Tests:            188410   Succeeded:                     0
----------------------------------------------------------------------------
----------------------------------------------------------------------------
Render Time:
  Photon Time:      No photons
  Radiosity Time:   No radiosity
  Trace Time:       0 hours  0 minutes  0 seconds (0.151 seconds)
              using 16 thread(s) with 1.932 CPU-seconds total
POV-Ray finished

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary

File 'new_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000002.pov' line 6: Parse Warning: Polygon not closed. Closing it.

...

Shadow Ray Tests:            188436   Succeeded:                     0
----------------------------------------------------------------------------
----------------------------------------------------------------------------
Render Time:
  Photon Time:      No photons
  Radiosity Time:   No radiosity
  Trace Time:       0 hours  0 minutes  0 seconds (0.144 seconds)
              using 16 thread(s) with 1.969 CPU-seconds total
POV-Ray finished

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Th

File 'new_000013.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000013.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000013.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000013.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000013.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000013.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000013.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000013.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000013.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000013.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000013.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000013.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000013.pov' line 6: Parse Warning: Polygon not closed. Closing it.

Rendered 250000 of 250000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 500 x 500
----------------------------------------------------------------------------
Pixels:           266000   Samples:          142317   Smpls/Pxl: 0.54
Rays:             408317   Saved:                 0   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Polygon                        7962872          876489     11.01
Bounding Box                  45375282        16309518     35.94
----------------------------------------------------------------------------


Shadow Ray Tests:            194557   Succeeded:                     0
----------------------------------------------------------------------------
----------------------------------------------------------------------------
Render Time:
  Photon Time:      No photons
  Radiosity Time:   No radiosity
  Trace Time:       0 hours  0 minutes  0 seconds (0.155 seconds)
              using 16 thread(s) with 1.973 CPU-seconds total
POV-Ray finished

povray: cannot open the user configuration file /home/hyrodium/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Th

File 'new_000014.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000014.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000014.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000014.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000014.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000014.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000014.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000014.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000014.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000014.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000014.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000014.pov' line 6: Parse Warning: Polygon not closed. Closing it.
File 'new_000014.pov' line 6: Parse Warning: Polygon not closed. Closing it.